In [1]:
import pandas as pd
import json
import os
from pathlib import Path
import datetime
from datetime import datetime
# Read the JSON files
feedback_dir = Path('./feedback')

# Initialize lists to store responses
child_responses = []
parent_responses = []
config_file = Path('./credentials.json')  # Adjust path as needed
with open(config_file, 'r') as f:
    config_data = json.load(f)


# Walk through the feedback directory
for user_dir in feedback_dir.iterdir():
    if user_dir.is_dir():
        username = user_dir.name
        
        # Process child responses - using glob to find all matching files
        child_files = list(user_dir.glob(f"child_{username}_*.json"))
        for child_file in child_files:
            try:
                with open(child_file, 'r') as f:
                    child_data = json.load(f)
                    child_data['username'] = username
                    child_responses.append(child_data)
            except json.JSONDecodeError as e:
                print(f"Error reading {child_file}: {e}")
                
        # Process parent responses - using glob to find all matching files
        parent_files = list(user_dir.glob(f"parent_{username}_*.json"))
        for parent_file in parent_files:
            try:
                with open(parent_file, 'r') as f:
                    parent_data = json.load(f)
                    parent_data['username'] = username
                    parent_data["parent_email"] = config_data[username]["parent_email"]
                    # Extract timestamp from filename
                    timestamp = parent_file.stem.split('_')[-2:]  # Gets the last part after splitting by _
                    parent_data['timestamp'] = timestamp
                    parent_responses.append(parent_data)
            except json.JSONDecodeError as e:
                print(f"Error reading {parent_file}: {e}")

# Create separate dataframes
df_children = pd.DataFrame(child_responses)
df_parents = pd.DataFrame(parent_responses)

# Sort by username and timestamp if they exist
if not df_children.empty:
    df_children = df_children.sort_values(['username'])
if not df_parents.empty:
    df_parents = df_parents.sort_values(['username'])

# Save to CSV (optional)
# timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
df_children.to_csv(f'child_responses.csv', index=False)
df_parents.to_csv(f'parent_responses.csv', index=False)